# Tutorial 6: Simple Surface Processes

In [ ]:
from underworld import UWGeodynamics as GEO
from underworld import visualisation as vis

import underworld.function as fn

In [ ]:
u = GEO.UnitRegistry

In [ ]:
# Characteristic values of the system
half_rate = 1.8 * u.centimeter / u.year
model_length = 360e3 * u.meter
model_height = 120e3 * u.meter
refViscosity = 1e24 * u.pascal * u.second
surfaceTemp = 273.15 * u.degK
baseModelTemp = 1603.15 * u.degK
bodyforce = 3300 * u.kilogram / u.metre**3 * 9.81 * u.meter / u.second**2

KL = model_length
Kt = KL / half_rate
KM = bodyforce * KL**2 * Kt**2
KT = (baseModelTemp - surfaceTemp)

GEO.scaling_coefficients["[length]"] = KL
GEO.scaling_coefficients["[time]"] = Kt
GEO.scaling_coefficients["[mass]"]= KM
GEO.scaling_coefficients["[temperature]"] = KT

In [ ]:
Model = GEO.Model(elementRes=(16, 16, 16), 
                  minCoord=(0. * u.kilometer, 0. * u.kilometer, -110. * u.kilometer), 
                  maxCoord=(120. * u.kilometer, 120. * u.kilometer, 10. * u.kilometer), 
                  gravity=(0.0, 0.0, -9.81 * u.meter / u.second**2))

In [ ]:
Model.outputDir="outputs_tutorial6.3_velSP_3D"

In [ ]:
Model.diffusivity = 1e-6 * u.metre**2 / u.second 
Model.capacity    = 1000. * u.joule / (u.kelvin * u.kilogram)

In [ ]:
air               = Model.add_material(name="Air", shape=GEO.shapes.Layer3D(top=Model.top, bottom=0.0 * u.kilometer))
# stickyAir         = Model.add_material(name="StickyAir", shape=GEO.shapes.Layer2D(top=air.bottom, bottom= 0.0 * u.kilometer))
uppercrust        = Model.add_material(name="UppperCrust", shape=GEO.shapes.Layer3D(top=air.bottom, bottom=-35.0 * u.kilometer))
mantleLithosphere = Model.add_material(name="MantleLithosphere", shape=GEO.shapes.Layer3D(top=uppercrust.bottom, bottom=-100.0 * u.kilometer))
mantle            = Model.add_material(name="Mantle", shape=GEO.shapes.Layer3D(top=mantleLithosphere.bottom, bottom=Model.bottom))
sediment          = Model.add_material(name="Sediment")

In [ ]:
air.diffusivity = 1.0e-6 * u.metre**2 / u.second
air.capacity = 100. * u.joule / (u.kelvin * u.kilogram)

# stickyAir.diffusivity = 1.0e-6 * u.metre**2 / u.second
# stickyAir.capacity = 100. * u.joule / (u.kelvin * u.kilogram)

In [ ]:
air.density                = 1. * u.kilogram / u.metre**3
# stickyAir.density          = 1. * u.kilogram / u.metre**3
uppercrust.density         = GEO.LinearDensity(reference_density=2620. * u.kilogram / u.metre**3)
mantleLithosphere.density  = GEO.LinearDensity(reference_density=3370. * u.kilogram / u.metre**3)
mantle.density             = GEO.LinearDensity(reference_density=3370. * u.kilogram / u.metre**3)
sediment.density           = GEO.LinearDensity(reference_density=2300. * u.kilogram / u.metre**3)

In [ ]:
uppercrust.radiogenicHeatProd = 0.7 * u.microwatt / u.meter**3
sediment.radiogenicHeatProd   = 0.7 * u.microwatt / u.meter**3
mantleLithosphere.radiogenicHeatProd = 0.02 * u.microwatt / u.meter**3

In [ ]:
rh = GEO.ViscousCreepRegistry()

In [ ]:
air.viscosity                = 1e19 * u.pascal * u.second
# stickyAir.viscosity          = 1e20 * u.pascal * u.second
uppercrust.viscosity         = 1 * rh.Wet_Quartz_Dislocation_Gleason_and_Tullis_1995
mantleLithosphere.viscosity  = rh.Dry_Olivine_Dislocation_Karato_and_Wu_1993
mantle.viscosity             = 0.2 * rh.Dry_Olivine_Dislocation_Karato_and_Wu_1993
sediment.viscosity           = rh.Wet_Quartz_Dislocation_Gleason_and_Tullis_1995

In [ ]:
plasticity = GEO.DruckerPrager(cohesion=20.0 * u.megapascal,
                               cohesionAfterSoftening=20 * u.megapascal,
                               frictionCoefficient=0.12,
                               frictionAfterSoftening=0.02,
                               epsilon1=0.5,
                               epsilon2=1.5)

In [ ]:
uppercrust.plasticity         = plasticity
mantleLithosphere.plasticity  = plasticity
mantle.plasticity             = plasticity
sediment.plasticity           = plasticity

## Temperature Boundary Condition

In [ ]:
Model.set_temperatureBCs(top=293.15 * u.degK, 
                         bottom=1603.15 * u.degK, 
                         materials=[(mantle, 1603.15 * u.degK), (air, 293.15 * u.degK)])

## Velocity Boundary Conditions

In [ ]:
vel = 2.5 * u.centimeter / u.year



vol_out = 2*(vel*(air.top - air.bottom)*Model.maxCoord[1]).to_base_units()
vol_out

In [ ]:
### Velocity at surface to replace air that gets removed at sides
vel_in = vol_out / (Model.maxCoord[0] * Model.maxCoord[1])
vel_in.to_base_units()

In [ ]:
Model.set_velocityBCs(left = [-vel, None, None],
                      right=[vel, None, None],
                      front=[None, 0.0, None], back=[None, 0.0, None],
                      top = [None, None, -1*vel_in],
                      bottom = [None, None, None])

## Initial Damage

In [ ]:
import numpy as np

def gaussian(xx, centre, width):
    return ( np.exp( -(xx - centre)**2 / width ))

maxDamage = 0.7
Model.plasticStrain.data[:] = 0.
Model.plasticStrain.data[:] = maxDamage * np.random.rand(*Model.plasticStrain.data.shape[:])
Model.plasticStrain.data[:,0] *= gaussian(Model.swarm.particleCoordinates.data[:,0], (GEO.nd(Model.maxCoord[0] - Model.minCoord[0])) / 2.0, GEO.nd(5.0 * u.kilometer))
Model.plasticStrain.data[:,0] *= gaussian(Model.swarm.particleCoordinates.data[:,2], GEO.nd(-35. * u.kilometer) , GEO.nd(5.0 * u.kilometer))

In [ ]:
if GEO.nProcs == 1:
    Fig = vis.Figure(resolution=(1200,600))
    Fig.Surface(Model.mesh, Model.plasticStrain, cullface=False, opacity=0.5)
    Fig.window()

In [ ]:
if GEO.nProcs == 1:
    Fig = vis.Figure(resolution=(1200,600))
    Fig.Surface(Model.mesh, Model.materialField, cullface=False, opacity=0.5)
    Fig.window()

### x and y coordinates for the surface

In [ ]:
import numpy as np

x = np.linspace(Model.minCoord[0], Model.maxCoord[0], 4*(Model.mesh.elementRes[0]+1))
y = np.linspace(Model.minCoord[1], Model.maxCoord[1], 4*(Model.mesh.elementRes[1]+1))

xi, yi = np.meshgrid(x, y)

coords = np.zeros(shape=(xi.flatten().shape[0], 3))
coords[:,0] = xi.flatten()
coords[:,1] = yi.flatten()
coords[:,2] = np.zeros_like(coords[:,0]) ### or any array with same shape as x and y coords with the initial height

### add back in the dim
coords = coords * u.kilometer

### Include erosion and sedimentation rates in model runs

A branching condition is used to create erosion and sedimentation rates that can vary across the domain

In [ ]:
ve_conditions = fn.branching.conditional([((Model.y >= GEO.nd(Model.maxCoord[1])/2.), GEO.nd(2.5 * u.millimeter/u.year)),
                                            (True, GEO.nd(0.0 * u.millimeter/u.year))])

vs_conditions = fn.branching.conditional([((Model.y >= GEO.nd(Model.maxCoord[1])/2.), GEO.nd(2.5 * u.millimeter/u.year)),
                                          (True, GEO.nd(0.0 * u.millimeter/u.year))])

Model.surfaceProcesses = GEO.surfaceProcesses.velocitySurface_3D(airIndex     = air.index,
                                                               sedimentIndex= sediment.index,
                                                               surfaceArray = coords,                ### grid with surface points (x, y, z)
                                                               vs_condition = vs_conditions,         ### sedimentation rate at each grid point
                                                               ve_condition = ve_conditions,         ### erosion rate at each grid point
                                                               surfaceElevation=air.bottom)

In [ ]:
from underworld import visualisation as vis
Fig = vis.Figure(figsize=(1200,400))
Fig.Points(Model.surface_tracers, Model.surface_tracers.ve, fn_size=5)
Fig.show()

In [ ]:
from underworld import visualisation as vis
Fig = vis.Figure(figsize=(1200,400))
Fig.Points(Model.surface_tracers, Model.surface_tracers.vs, fn_size=5)
Fig.show()

In [ ]:
Model.init_model(temperature="steady-state", pressure="lithostatic")

In [ ]:
GEO.rcParams['initial.nonlinear.min.iterations'] = 1
GEO.rcParams['nonlinear.min.iterations'] = 1

In [ ]:
# Model.solver.set_inner_method("mumps")
# Model.solver.set_penalty(1e6)
GEO.rcParams["initial.nonlinear.tolerance"] = 1e-2

In [ ]:
Model.run_for(duration=0.51 * u.megayear, checkpoint_interval=0.5*u.megayears)

In [ ]:
if GEO.size == 1:
    import matplotlib.pyplot as plt
    
    
    surface = GEO.dim(Model.surface_tracers.data, u.kilometer)

    scatter = plt.scatter(surface[:,0], surface[:,1], c=surface[:,2], s=10)

    cbar = plt.colorbar(scatter)
    
    cbar.set_label('Topo [km]')
    
    
    plt.xlabel('x [km]')
    plt.xlabel('y [km]')
    
    plt.show()
    
    
    plt.plot()
                  
    profile1 = surface[surface[:,1].m == np.unique(surface.m[:,0])[20]]
    profile2 = surface[surface[:,1].m == np.unique(surface.m[:,0])[-20]]
                        
    plt.plot(profile1[:,0], profile1[:,2], label = 'No SP') 
                          
    plt.plot(profile2[:,0], profile2[:,2], label = 'SP')
    
    plt.xlabel('x [km]')
    
    plt.ylabel('Topo [km]')
    
    plt.legend()
    
    